In [4]:
import time
import pandas as pd
from io import StringIO

import requests
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent / 'scripts'))

print(sys.path)

from _helpers import to_date_period, to_datetime
from _elexon_helpers import robust_request


['c:\\Users\\lukas\\OneDrive\\Desktop\\projects\\nodal\\UKLMP\\notebooks', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\python310.zip', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\DLLs', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\lib', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur', '', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\lib\\site-packages', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\lib\\site-packages\\win32', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\lib\\site-packages\\win32\\lib', 'c:\\Users\\lukas\\miniforge3\\envs\\pypsa-eur\\lib\\site-packages\\Pythonwin', 'c:\\Users\\lukas\\OneDrive\\Desktop\\projects\\nodal\\UKLMP\\scripts', 'c:\\Users\\lukas\\OneDrive\\Desktop\\projects\\nodal\\UKLMP\\scripts']


In [2]:
pn_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/PN?settlementDate={}&settlementPeriod={}&format=csv"

In [5]:
date = '2020-06-01'
period = 20

response = robust_request(requests.get, pn_url.format(date, period))
df = pd.read_csv(StringIO(response.text))

df.TimeFrom.value_counts().iloc[[0]]

TimeFrom
2020-06-01T08:30:00Z    1189
Name: count, dtype: int64

In [32]:
# df.to_csv('pn_example.csv')
h = df.copy()

h['TimeFrom'] = pd.to_datetime(h['TimeFrom'])
h['TimeTo'] = pd.to_datetime(h['TimeTo'])

# df['length'] -

h['TimeLength'] = h['TimeTo'] - h['TimeFrom']
h['AverageLevel'] = (
    h[['LevelFrom', 'LevelTo']]
    .mean(axis=1)
    .mul(
        h['TimeLength'].div(pd.Timedelta('30min'))
        )
    )

(
    h.groupby('NationalGridBmUnit')
    ["AverageLevel"].sum()
    .rename(pd.Timestamp(to_datetime(date, period)))
)



NationalGridBmUnit
SSC-LON01   -1461.0
LON-LON01   -1416.5
STH-LON01   -1347.5
SEE-LON01    -948.5
SWT-LON01    -758.0
              ...  
SIZB-1        602.0
HEYM27        621.0
DRAXX-2       645.0
TORN-2        652.0
ISG-NDPL1     694.0
Name: AverageLevel, Length: 1427, dtype: float64

In [56]:
h.groupby('NationalGridBmUnit')["TimeLength"].sum().value_counts()

TimeLength
0 days 00:30:00    1427
Name: count, dtype: int64

In [48]:
period = 2
start = to_datetime(date, period)
print(start)
end = start + pd.Timedelta("30min")

prep_time = lambda x: str(x).zfill(2)

mels_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/MELS?from={}T{}%3A{}Z&to={}T{}%3A{}Z&format=csv"

2022-02-01 00:30:00


In [51]:
print(
    start.strftime('%Y-%m-%d'),
    prep_time(start.hour),
    prep_time(start.minute),
    # end.strftime('%Y-%m-%d'),
    # prep_time(end.hour),
    # prep_time(end.minute)
    start.strftime('%Y-%m-%d'),
    prep_time(start.hour),
    prep_time(start.minute)
)

response = robust_request(
    requests.get,
    mels_url.format(
        start.strftime('%Y-%m-%d'),
        prep_time(start.hour),
        prep_time(start.minute),
        start.strftime('%Y-%m-%d'),
        prep_time(start.hour),
        prep_time(start.minute)
    )
)

2022-02-01 00 30 2022-02-01 00 30


In [71]:
df = pd.read_csv(StringIO(response.text))
df.TimeFrom.value_counts()

TimeFrom
2022-02-01T00:30:00Z    1427
Name: count, dtype: int64

In [106]:

date, period = '2020-06-01', 20
dt = to_datetime(date, period)
print(dt)

to_date_period(dt)

2020-06-01 08:30:00+00:00


('2020-06-01', 20)

In [94]:
# pd.Timestamp('2020-01-01').tz is None


True

In [72]:
df['TimeFrom'] = pd.to_datetime(df['TimeFrom'])
df['TimeTo'] = pd.to_datetime(df['TimeTo'])

# df['TimeLength'] = df['TimeTo'] - df['TimeFrom']

In [73]:
# df.groupby('NationalGridBmUnit')["TimeLength"].sum().value_counts()
# df.TimeTo.value_counts()
# ss = df.loc[df.TimeTo < '2022-02-01 01:00:00'].set_index('NationalGridBmUnit')

ss = df.loc[df.TimeTo < end].set_index('NationalGridBmUnit')

TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

In [69]:
d = df.groupby('NationalGridBmUnit')["TimeTo"].max()
d = d.loc[d < '2022-02-01 01:00:00']

# hold = df.loc[df.NationalGridBmUnit.isin(d.index)]

df.loc[df.NationalGridBmUnit.isin(d.index), 'TimeTo'] = pd.Timestamp('2022-02-01 01:00:00', tz='UTC')

df['TimeLength'] = df['TimeTo'] - df['TimeFrom']


In [70]:
df.groupby('NationalGridBmUnit')["TimeLength"].sum().value_counts()

TimeLength
0 days 00:30:00    1427
Name: count, dtype: int64